In [76]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
#import time

import random as random
import gmaps
from pprint import pprint

#import pycountry
#from IPython.display import (Image, HTML)
#import logging
#import math
#from scipy import stats

# Functions to find the distance between 2 lat/long coordinates
# Description: https://pypi.org/project/geopy/1.9.1/
from geopy.distance import (distance, great_circle)

# Keys
from api_config import (key_openweathermap, key_gmaps)

# Incorporated citipy to determine city based on latitude and longitude
#from citipy import citipy


In [77]:
# FYI: Function zipcode_from_latlong()
# A function to use reverse geocode lookup to find a
#  postal_code (zipcode) associated with a lat/long coord

def zipcode_from_latlong( a_lat, a_long ):
    baseurl = "https://maps.googleapis.com/maps/api/geocode/json?"
    latlong = f"latlng={a_lat},{a_long}"
    api_key = f"&key={key_gmaps}"
    
    full_url = baseurl + latlong + api_key

    # Perform a reverse geocode loopup to find the zipcode associated with this lat/long coord
    g_response = requests.get(full_url)
    g_json = g_response.json()
    
    # Traverse the results to find a zipcode for this address
    zipcode = None
    for a in r_json['results'][0]['address_components']:
        if 'postal_code' in a['types']:
            zipcode = a['long_name']
            
    # Return the zipcode that was found
    return zipcode

# Data sources for analysis
* Chicago Transit Authority: https://www.transitchicago.com/developers/
    * CTA Transit Stops - General Transit Feed Specification (GTFS): "Data/access-Chicago-CTA-stops.txt"

In [78]:
# Read in the Chicago CTA stop + zipcode info previously cleaned
i_file = "../Data/chicago_cta_stops.csv"

# Read the data into a dataframe
c_cta_stops_df = pd.read_csv(i_file)

# Data types
# Note: 'stop_code' and 'parent_station' both are type 'float64' since these columns may have NaN values.
# (For some reason, NaN results in an error if the data type is 'int64')
c_cta_stops_df.dtypes

# Quick preview of the data
c_cta_stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,postal_code
0,1,1.0,Jackson & Austin Terminal,"Jackson & Austin Terminal, Northeastbound, Bus...",41.876322,-87.774105,0,NaN,1,60644
1,2,2.0,5900 W Jackson,"5900 W Jackson, Eastbound, Southside of the St...",41.877067,-87.771318,0,NaN,1,60644
2,3,3.0,Jackson & Menard,"Jackson & Menard, Eastbound, Southside of the ...",41.876957,-87.769750,0,NaN,1,60644
3,4,4.0,5700 W Jackson,"5700 W Jackson, Eastbound, Southside of the St...",41.877024,-87.767451,0,NaN,1,60644
4,6,6.0,Jackson & Lotus,"Jackson & Lotus, Eastbound, Southeast Corner",41.876513,-87.761446,0,NaN,1,60644


In [79]:
c_cta_stops_df[ 'stop_name' ].value_counts().head()

35th/Archer Orange Line Station    8
Midway Orange Line Station         8
Paulina & Howard Terminal          6
Kedzie Orange Line Station         6
69th Red Line Station              5
Name: stop_name, dtype: int64

In [80]:
c_cta_stops_df.loc[ c_cta_stops_df[ 'stop_name' ] == '35th/Archer Orange Line Station' ]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,postal_code
3009,4432,4432.0,35th/Archer Orange Line Station,"35th/Archer Orange Line Station, Southwestboun...",41.828732,-87.681035,0,NaN,1,60644
8795,14133,14133.0,35th/Archer Orange Line Station,"35th/Archer Orange Line Station, Southwestboun...",41.828688,-87.680989,0,NaN,1,60644
8796,14134,14134.0,35th/Archer Orange Line Station,"35th/Archer Orange Line Station, Southwestboun...",41.828769,-87.681079,0,NaN,1,60644
8797,14135,14135.0,35th/Archer Orange Line Station,"35th/Archer Orange Line Station, Southwestboun...",41.828818,-87.681127,0,NaN,1,60644
8798,14136,14136.0,35th/Archer Orange Line Station,"35th/Archer Orange Line Station, Southwestboun...",41.828859,-87.681171,0,NaN,1,60644
9714,15683,15683.0,35th/Archer Orange Line Station,"35th/Archer Orange Line Station, Northeastboun...",41.828905,-87.681216,0,NaN,1,60644
10307,17435,17435.0,35th/Archer Orange Line Station,"35th/Archer Orange Line Station, Southwestboun...",41.828655,-87.680955,0,NaN,1,60644
10352,17509,17509.0,35th/Archer Orange Line Station,NaN,41.829267,-87.680527,0,NaN,1,60644


In [81]:
for ci in c_cta_stops_df.index:
    if c_cta_stops_df.loc[ci, 'stop_id'] == 4432:
            print (ci)
            break

3009


In [82]:
ac = ( c_cta_stops_df.loc[3009, 'stop_lat'], c_cta_stops_df.loc[3009, 'stop_lon'])
bc = ( c_cta_stops_df.loc[8795, 'stop_lat'], c_cta_stops_df.loc[8795, 'stop_lon'])
print (ac)
print (bc)

(41.8287319, -87.68103544)
(41.82868757, -87.6809893)


In [84]:
dist_v = distance(ac, bc).feet
print (dist_v)
print (dist_g)

20.471699711787462
20.46596452114071


In [86]:
tempDataList = [(39.7612992, -86.1519681), 
                (39.762241,  -86.158436 ), 
                (39.7622292, -86.1578917)]

def closest(data, y):
    retval = min( data, key=lambda z: distance( z, y ).feet )
    return retval

r = (39.7622290, -86.1519750)

closest(tempDataList, r)

(39.7612992, -86.1519681)

In [172]:
# TO DO THIS LATER!

# Function to find the CTA stop closest to the reference point provided
# r: reference point as a tuple (lat, long)
# stop_coords: a list of tuples with ('stop_lat', 'stop_lon')
#               generated from the dataframe containing CTA stops

def closest_stop(stop_coords, r):
    # Find the lat/long tuple closest to the reference point provided
    close_point = min( stop_coords, key=lambda z: distance( z, r ).feet )
    
    # Get the index of this closest point
    # (Note, if there are dups in the list just return the first index)
    retval = stop_coords.index( close_point )
    return retval

# Generate the list of CTA stop coordinates
stop_coords = list( zip( c_cta_stops_df['stop_lat'], c_cta_stops_df['stop_lon'] ) )

# A reference point for which we want to find the index of the closest CTA stop
refpt = (41.87659462, -87.75461525)

# Index of the closest CTA stop
closest_stop_index = closest_stop( stop_coords, refpt)

c_cta_stops_df.loc[closest_stop_index,'postal_code']

# Group and Merge for Analysis
## Comparison of ratings and transit stop density on per-zipcode basis

In [127]:
# Read in zip code data (which will provide lat/long for each zip code)
i_file = "../Data/zip_code_database.csv"
zip_info_df = pd.read_csv(i_file)
zip_info_df.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [ ]:
i_file = "../Raw Data/US_Zip_Codes_from_2013_Government_Data.csv"
zip_latlong_df = pd.read_csv(i_file)

In [256]:
zip_c_only_df = (zip_latlong_df.loc[ zip_latlong_df['ZIP'] > 60000 ]).loc[ zip_latlong_df['ZIP'] < 61000 ]
zip_c_only_df.reset_index(drop=True, inplace=True)

In [257]:
zip_c_only_df

,ZIP,LAT,LNG
0,60002,42.471741,-88.084493
1,60004,42.112780,-87.979542
2,60005,42.064490,-87.985462
3,60007,42.008600,-87.997340
4,60008,42.069786,-88.016221
5,60010,42.146494,-88.164651
6,60012,42.272492,-88.314084
7,60013,42.223439,-88.235506
8,60014,42.232414,-88.327449
9,60015,42.173809,-87.878173


In [128]:
# Read in yelp data will will provide restaurant ratings, cuisine, lat/long, etc.
i_file = "../Data/Yelp_Restaurants_Chicago.csv"
rest_zip_df = pd.read_csv(i_file)
rest_zip_df.head()

,zip,city,state,name,price,rating,review_count,type,latitude,longitude
0,60601,Chicago,IL,Wildberry Pancakes and Cafe,$$,4.5,5862,American (New),41.884668,-87.622880
1,60603,Chicago,IL,The Gage,$$,4.0,2632,American (New),41.881048,-87.624533
2,60603,Chicago,IL,Cindy's,$$,4.0,1387,American (New),41.881689,-87.625006
3,60603,Chicago,IL,The Marq,$$,4.0,509,American (New),41.879619,-87.629925
4,60603,Chicago,IL,Cherry Circle Room,$$$,4.5,330,American (New),41.881664,-87.624969


In [139]:
# Chicago CTA stops data
c_cta_stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,postal_code
0,1,1.0,Jackson & Austin Terminal,"Jackson & Austin Terminal, Northeastbound, Bus...",41.876322,-87.774105,0,NaN,1,60644
1,2,2.0,5900 W Jackson,"5900 W Jackson, Eastbound, Southside of the St...",41.877067,-87.771318,0,NaN,1,60644
2,3,3.0,Jackson & Menard,"Jackson & Menard, Eastbound, Southside of the ...",41.876957,-87.769750,0,NaN,1,60644
3,4,4.0,5700 W Jackson,"5700 W Jackson, Eastbound, Southside of the St...",41.877024,-87.767451,0,NaN,1,60644
4,6,6.0,Jackson & Lotus,"Jackson & Lotus, Eastbound, Southeast Corner",41.876513,-87.761446,0,NaN,1,60644


# Fix the zip codes in the CTA Stop data

In [198]:
# Function to find the CTA stop closest to the reference point provided
# r: reference point as a tuple (lat, long)
# stop_coords: a list of tuples with ('stop_lat', 'stop_lon')
#               generated from the dataframe containing CTA stops

def closest_coord(stop_coords, r):
    # Find the lat/long tuple closest to the reference point provided
    close_point = min( stop_coords, key=lambda z: distance( z, r ).feet )
    
    # Get the index of this closest point
    # (Note, if there are dups in the list just return the first index)
    retval = stop_coords.index( close_point )
    return retval




In [ ]:
# Generate the list of CTA stop coordinates
#stop_coords = list( zip( c_cta_stops_df['stop_lat'], c_cta_stops_df['stop_lon'] ) )

In [ ]:
# Index of the closest CTA stop
# closest_stop_index = closest_coord( stop_coords, refpt)
# c_cta_stops_df.loc[closest_stop_index,'postal_code']

In [258]:
# Generate the list of zipcode coordinates
zip_coords = list( zip( zip_c_only_df['LAT'], zip_c_only_df['LNG'] ) )
zip_coords

[(42.471740999999994, -88.08449300000001),
 (42.11278, -87.97954200000001),
 (42.06449, -87.985462),
 (42.0086, -87.99734000000001),
 (42.069786, -88.016221),
 (42.146494, -88.164651),
 (42.272492, -88.314084),
 (42.223439, -88.235506),
 (42.232414, -88.327449),
 (42.173809000000006, -87.878173),
 (42.049572999999995, -87.895003),
 (41.979388, -87.894358),
 (42.389053999999994, -88.173346),
 (42.194697, -88.217108),
 (42.135422999999996, -87.761882),
 (42.075201, -87.821026),
 (42.092234000000005, -87.837445),
 (42.059253000000005, -87.778339),
 (42.337078000000005, -88.044326),
 (42.373574, -87.939353),
 (42.424786, -88.608011),
 (42.456317999999996, -88.41969499999999),
 (42.185968, -87.805939),
 (42.206363, -87.813098),
 (42.364552, -88.15111999999999),
 (42.278206, -88.196627),
 (42.088859, -87.714588),
 (42.284408, -87.862297),
 (42.235997, -87.864777),
 (42.416323, -88.059119),
 (42.203134000000006, -88.044208),
 (42.295408, -87.950611),
 (42.331146999999994, -88.29518),
 (42.354

In [259]:
refpt = ( c_cta_stops_df['stop_lat'][0], c_cta_stops_df['stop_lon'][0] )
refpt

(41.87632184, -87.77410482)

In [260]:
closest_zip_index = closest_coord( zip_coords, refpt)
closest_zip_index

140

In [261]:
closest_zip = zip_c_only_df.loc[closest_zip_index,'ZIP']
closest_zip

60304

In [267]:
for row in c_cta_stops_df.itertuples():
    stop_id = getattr(row, 'stop_id')
    stop_name = getattr(row, 'stop_name')
    postal_code = getattr(row, 'postal_code')

    si = getattr(row, 'Index')
    refpt = ( getattr(row, 'stop_lat'), getattr(row, 'stop_lon') )
    closest_zip_index = closest_coord( zip_coords, refpt)
    closest_zip = zip_c_only_df.loc[closest_zip_index,'ZIP']
    
    # Fix the postal code
    c_cta_stops_df['postal_code'][si] = closest_zip
    
    print(f"Index {si}: CTA Stop:[{stop_id}] {stop_name} - Postal Code:{postal_code} => New Postal Code: {closest_zip}")
    #print(f">> via direct indexing: CTA Stop:[{c_cta_stops_df['stop_id'][si]}] {c_cta_stops_df['stop_name'][si]} - Postal Code:{c_cta_stops_df['postal_code'][si]}")
    #print(f"Index {si}: Postal Code:{getattr(row, 'postal_code')} => New Postal Code: {closest_zip}")

C:\Users\Jeff\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Index 0: CTA Stop:[1] Jackson & Austin Terminal - Postal Code:60644 => New Postal Code: 60304
Index 1: CTA Stop:[2] 5900 W Jackson - Postal Code:60644 => New Postal Code: 60644
Index 2: CTA Stop:[3] Jackson & Menard - Postal Code:60644 => New Postal Code: 60644
Index 3: CTA Stop:[4] 5700 W Jackson - Postal Code:60644 => New Postal Code: 60644
Index 4: CTA Stop:[6] Jackson & Lotus - Postal Code:60644 => New Postal Code: 60644
Index 5: CTA Stop:[7] 5351 W Jackson - Postal Code:60644 => New Postal Code: 60644
Index 6: CTA Stop:[8] Jackson & Lockwood - Postal Code:60644 => New Postal Code: 60644
Index 7: CTA Stop:[9] Jackson & Laramie - Postal Code:60644 => New Postal Code: 60644
Index 8: CTA Stop:[10] Jackson & Leamington - Postal Code:60644 => New Postal Code: 60644
Index 9: CTA Stop:[11] 5047 W Jackson - Postal Code:60644 => New Postal Code: 60644
Index 10: CTA Stop:[12] Jackson & Lavergne - Postal Code:60644 => New Postal Code: 60644
Index 11: CTA Stop:[13] 4849 W Jackson - Postal Code

Index 93: CTA Stop:[135] Jackson & Kostner - Postal Code:60644 => New Postal Code: 60624
Index 94: CTA Stop:[136] Jackson & Kilbourn - Postal Code:60644 => New Postal Code: 60624
Index 95: CTA Stop:[137] Jackson & Kolmar - Postal Code:60644 => New Postal Code: 60624
Index 96: CTA Stop:[138] Jackson & Kenton - Postal Code:60644 => New Postal Code: 60644
Index 97: CTA Stop:[139] Jackson & Kilpatrick - Postal Code:60644 => New Postal Code: 60644
Index 98: CTA Stop:[140] Jackson & Cicero - Postal Code:60644 => New Postal Code: 60644
Index 99: CTA Stop:[141] 4900 W Jackson - Postal Code:60644 => New Postal Code: 60644
Index 100: CTA Stop:[142] Jackson & Lavergne - Postal Code:60644 => New Postal Code: 60644
Index 101: CTA Stop:[143] 5056 W Jackson - Postal Code:60644 => New Postal Code: 60644
Index 102: CTA Stop:[144] Jackson & Leamington - Postal Code:60644 => New Postal Code: 60644
Index 103: CTA Stop:[145] Jackson & Laramie - Postal Code:60644 => New Postal Code: 60644
Index 104: CTA Sto

Index 183: CTA Stop:[250] 5400 W Harrison - Postal Code:60644 => New Postal Code: 60644
Index 184: CTA Stop:[252] Harrison & Central - Postal Code:60644 => New Postal Code: 60644
Index 185: CTA Stop:[257] 5500 W Roosevelt - Postal Code:60644 => New Postal Code: 60644
Index 186: CTA Stop:[258] Roosevelt & 54th Avenue - Postal Code:60644 => New Postal Code: 60644
Index 187: CTA Stop:[259] 5300 W. Roosevelt - Postal Code:60644 => New Postal Code: 60644
Index 188: CTA Stop:[260] Roosevelt & Laramie - Postal Code:60644 => New Postal Code: 60644
Index 189: CTA Stop:[261] Roosevelt & 51st Avenue - Postal Code:60644 => New Postal Code: 60644
Index 190: CTA Stop:[262] Roosevelt & 50th Avenue - Postal Code:60644 => New Postal Code: 60644
Index 191: CTA Stop:[263] Roosevelt & 49th Avenue - Postal Code:60644 => New Postal Code: 60644
Index 192: CTA Stop:[265] Roosevelt & 47th Avenue - Postal Code:60644 => New Postal Code: 60644
Index 193: CTA Stop:[268] Roosevelt & Kilbourn - Postal Code:60644 => 

Index 272: CTA Stop:[406] Madison & Kildare - Postal Code:60644 => New Postal Code: 60624
Index 273: CTA Stop:[409] Madison & Pulaski - Postal Code:60644 => New Postal Code: 60624
Index 274: CTA Stop:[412] Madison & Central Park - Postal Code:60644 => New Postal Code: 60624
Index 275: CTA Stop:[413] Madison & St. Louis - Postal Code:60644 => New Postal Code: 60624
Index 276: CTA Stop:[415] Madison & Spaulding - Postal Code:60644 => New Postal Code: 60624
Index 277: CTA Stop:[416] Madison & Kedzie - Postal Code:60644 => New Postal Code: 60624
Index 278: CTA Stop:[417] Madison & Albany - Postal Code:60644 => New Postal Code: 60612
Index 279: CTA Stop:[420] Madison & California - Postal Code:60644 => New Postal Code: 60612
Index 280: CTA Stop:[421] Madison & Washtenaw - Postal Code:60644 => New Postal Code: 60612
Index 281: CTA Stop:[423] Madison & Campbell - Postal Code:60644 => New Postal Code: 60612
Index 282: CTA Stop:[425] Madison & Oakley - Postal Code:60644 => New Postal Code: 6061

...

Index 10613: CTA Stop:[18009] Halsted & Jackson - Postal Code:60644 => New Postal Code: 60607
Index 10614: CTA Stop:[18010] Broadway & Foster - Postal Code:60644 => New Postal Code: 60640
Index 10615: CTA Stop:[18011] Halsted & Randolph - Postal Code:60644 => New Postal Code: 60661
Index 10616: CTA Stop:[18012] 5800 S Ellis - Postal Code:60644 => New Postal Code: 60637
Index 10617: CTA Stop:[18015] Dodge & Dempster - Postal Code:60644 => New Postal Code: 60202
Index 10618: CTA Stop:[18016] State & 94th Street - Postal Code:60644 => New Postal Code: 60619
Index 10619: CTA Stop:[18018] 55th Street & University - Postal Code:60644 => New Postal Code: 60615
Index 10620: CTA Stop:[18019] Pershing & Ashland - Postal Code:60644 => New Postal Code: 60609
Index 10621: CTA Stop:[18020] Ashland & Pershing - Postal Code:60644 => New Postal Code: 60609
Index 10622: CTA Stop:[18022] 51st Street & Loomis - Postal Code:60644 => New Postal Code: 60609
Index 10623: CTA Stop:[18023] 51st Street & Racine 

Index 10698: CTA Stop:[18174] Roosevelt & Jefferson - Postal Code:60644 => New Postal Code: 60607
Index 10699: CTA Stop:[18175] Homan & Chicago - Postal Code:60644 => New Postal Code: 60624
Index 10700: CTA Stop:[18176] Homan & Chicago - Postal Code:60644 => New Postal Code: 60624
Index 10701: CTA Stop:[18179] 133rd Place & Langley - Postal Code:60644 => New Postal Code: 60827
Index 10702: CTA Stop:[18180] Ashland & Marquette Rd - Postal Code:60644 => New Postal Code: 60636
Index 10703: CTA Stop:[18181] Ashland & 71st Street - Postal Code:60644 => New Postal Code: 60636
Index 10704: CTA Stop:[18184] UIC-Halsted Blue Line Station - Postal Code:60644 => New Postal Code: 60607
Index 10705: CTA Stop:[18185] Cicero & 55th Street - Postal Code:60644 => New Postal Code: 60638
Index 10706: CTA Stop:[18204] 35th Street & Ashland - Postal Code:60644 => New Postal Code: 60608
Index 10707: CTA Stop:[18205] 35th Street & Morgan - Postal Code:60644 => New Postal Code: 60609
Index 10708: CTA Stop:[18

Index 10784: CTA Stop:[18319] Fullerton & Greenview - Postal Code:60644 => New Postal Code: 60614
Index 10785: CTA Stop:[18320] Kedzie & Pratt - Postal Code:60644 => New Postal Code: 60645
Index 10786: CTA Stop:[18321] Damen & Fullerton - Postal Code:60644 => New Postal Code: 60647
Index 10787: CTA Stop:[18322] Ridge & Mulford - Postal Code:60644 => New Postal Code: 60202
Index 10788: CTA Stop:[18323] Grand & Chicago - Postal Code:60644 => New Postal Code: 60622
Index 10789: CTA Stop:[18329] Diversey & Clark/Broadway - Postal Code:60644 => New Postal Code: 60657
Index 10790: CTA Stop:[18330] Western & Elston/Diversey - Postal Code:60644 => New Postal Code: 60647
Index 10791: CTA Stop:[18331] Western & Belmont - Postal Code:60644 => New Postal Code: 60618
Index 10792: CTA Stop:[18333] 79th Street & King Drive - Postal Code:60644 => New Postal Code: 60619
Index 10793: CTA Stop:[18334] Harrison & Ashland - Postal Code:60644 => New Postal Code: 60607
Index 10794: CTA Stop:[18335] Harrison 

Index 10872: CTA Stop:[40130] 51st - Postal Code:60644 => New Postal Code: 60615
Index 10873: CTA Stop:[40140] Dempster-Skokie - Postal Code:60644 => New Postal Code: 60077
Index 10874: CTA Stop:[40150] Pulaski (Pink) - Postal Code:60644 => New Postal Code: 60623
Index 10875: CTA Stop:[40160] LaSalle/Van Buren - Postal Code:60644 => New Postal Code: 60604
Index 10876: CTA Stop:[40170] Ashland (Green/Pink) - Postal Code:60644 => New Postal Code: 60607
Index 10877: CTA Stop:[40180] Oak Park (Blue) - Postal Code:60644 => New Postal Code: 60304
Index 10878: CTA Stop:[40190] Sox-35th - Postal Code:60644 => New Postal Code: 60616
Index 10879: CTA Stop:[40210] Damen (Pink) - Postal Code:60644 => New Postal Code: 60608
Index 10880: CTA Stop:[40220] Western (Blue - Forest Park Branch) - Postal Code:60644 => New Postal Code: 60612
Index 10881: CTA Stop:[40230] Cumberland - Postal Code:60644 => New Postal Code: 60656
Index 10882: CTA Stop:[40240] 79th - Postal Code:60644 => New Postal Code: 60619

Index 10963: CTA Stop:[41130] Halsted (Orange) - Postal Code:60644 => New Postal Code: 60608
Index 10964: CTA Stop:[41140] King Drive - Postal Code:60644 => New Postal Code: 60637
Index 10965: CTA Stop:[41150] Kedzie (Orange) - Postal Code:60644 => New Postal Code: 60632
Index 10966: CTA Stop:[41160] Clinton (Green/Pink) - Postal Code:60644 => New Postal Code: 60661
Index 10967: CTA Stop:[41170] Garfield (Red) - Postal Code:60644 => New Postal Code: 60621
Index 10968: CTA Stop:[41180] Kedzie (Brown) - Postal Code:60644 => New Postal Code: 60625
Index 10969: CTA Stop:[41190] Jarvis - Postal Code:60644 => New Postal Code: 60626
Index 10970: CTA Stop:[41200] Argyle - Postal Code:60644 => New Postal Code: 60640
Index 10971: CTA Stop:[41210] Wellington (Brown/Purple) - Postal Code:60644 => New Postal Code: 60657
Index 10972: CTA Stop:[41220] Fullerton - Postal Code:60644 => New Postal Code: 60614
Index 10973: CTA Stop:[41230] 47th (Red) - Postal Code:60644 => New Postal Code: 60609
Index 10

In [268]:
c_cta_stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,postal_code
0,1,1.0,Jackson & Austin Terminal,"Jackson & Austin Terminal, Northeastbound, Bus...",41.876322,-87.774105,0,NaN,1,60304
1,2,2.0,5900 W Jackson,"5900 W Jackson, Eastbound, Southside of the St...",41.877067,-87.771318,0,NaN,1,60644
2,3,3.0,Jackson & Menard,"Jackson & Menard, Eastbound, Southside of the ...",41.876957,-87.769750,0,NaN,1,60644
3,4,4.0,5700 W Jackson,"5700 W Jackson, Eastbound, Southside of the St...",41.877024,-87.767451,0,NaN,1,60644
4,6,6.0,Jackson & Lotus,"Jackson & Lotus, Eastbound, Southeast Corner",41.876513,-87.761446,0,NaN,1,60644


In [269]:
o_file = "../Data/corrected_chicago_cta_stops.csv"
c_cta_stops_df.to_csv(o_file, index=False)

## Calculate the total number of CTA stops per zip code

In [276]:
c_cta_zip_g = c_cta_stops_df.groupby('postal_code')
c_cta_zip_g.agg('count')

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding
postal_code,,,,,,,,,
60018,1,0,1,0,1,1,1,0,1
60029,15,15,15,14,15,15,15,0,15
60053,1,1,1,1,1,1,1,0,1
60068,10,10,10,10,10,10,10,0,10
60076,27,27,27,27,27,27,27,0,27
60077,67,65,67,65,67,67,67,0,67
60091,18,18,18,3,18,18,18,0,18
60130,2,0,2,0,2,2,2,0,2
60171,12,12,12,12,12,12,12,0,12


## Start aggregates of all restaurant types within the zipcode